In [4]:
from google.colab import drive
import pandas as pd
import os

# Mount Google Drive
drive.mount("/content/drive")

# Define path to CSV file
base_path = "/content/drive/My Drive/WebKnoGraph/data"
file_path = os.path.join(base_path, "url_analysis_results.csv")


def get_worst_candidates(file_path, folder_depth_level, n_worst):
    df = pd.read_csv(file_path)
    required_cols = {"URL", "Folder_Depth", "PageRank"}
    if not required_cols.issubset(df.columns):
        raise ValueError(f"Missing columns: {required_cols - set(df.columns)}")
    filtered_df = df[df["Folder_Depth"] == folder_depth_level]
    sorted_df = filtered_df.sort_values(by="PageRank", ascending=True)
    worst_candidates = sorted_df.head(n_worst)
    return worst_candidates[["URL"]]


# Inputs
folder_depth_input = int(input("Enter folder depth level (integer): "))
n_worst_input = int(input("Enter number of worst candidates to retrieve: "))

# Run
worst_pages = get_worst_candidates(file_path, folder_depth_input, n_worst_input)
for url in worst_pages["URL"]:
    print(url)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter folder depth level (integer): 3
Enter number of worst candidates to retrieve: 200
https://kalicube.com/learning-spaces/faq/dont-let-your-brand-fall-behind-secure-your-place-in-the-digital-spotlight/
https://kalicube.com/learning-spaces/created-by-ai/jason-barnard-and-the-genesis-of-the-kalicube-process/
https://kalicube.com/learning-spaces/faq/knowledge-panels/
https://kalicube.com/learning-spaces/faq/brand-serps/
https://kalicube.com/the-kalicube-process-case-studies/personal-brand/losing-deals-to-digital-confusion-how-alex-morgan-used-the-kalicube-process-to-turn-ai-misrepresentation-into-1-2m-in-new-business/
https://kalicube.com/case-studies/brand-serp/changing-a-personal-brand-to-reflect-a-career-pivot/
https://kalicube.com/learning-spaces/faq/personal-brands/
https://kalicube.com/learning-spaces/faq/personal-branding-a-guide-from-kalicube/
https:/